In [ ]:
import os
from ln_utils import *

# Load data

In [ ]:
data_dir = "../LNdata/lncaptures/lngraph/2019/"
files =  [f for f in sorted(os.listdir(data_dir)) if ".json" in f]

In [ ]:
idx = 0
day_threshold = 7
snapshot_ends = [1549032366] # February 1, 2019 2:46:06 PM
for f in files[1:]:
    t = int(f.split(".")[0])
    diff = (t-snapshot_ends[idx])//86400
    if diff >= day_threshold:
        snapshot_ends.append(t)
        idx += 1

In [ ]:
graph_files = []

In [ ]:
dir_1 = "../LNdata/lncaptures/lngraph/2019/"
for time in snapshot_ends:
    graph_files.append("%s/%i.json" % (dir_1, time))

In [ ]:
dir_2 = "../LNdata/"
for time in ["0323","0329"]:
    graph_files.append("%s/%s.json" % (dir_2, time))

In [ ]:
graph_files

In [ ]:
EDGE_KEYS = ["node1_pub","node2_pub","last_update","capacity","channel_id",'node1_policy','node2_policy']
nodes, edges = load_temp_data(graph_files[:-1], edge_keys=EDGE_KEYS)
print(len(nodes), len(edges))

edges = edges.sort_values("last_update").reset_index(drop=True)
print(edges.head(3))

In [ ]:
edges["snapshot_id"].value_counts()

# Transform undirected graph into directed graph

In [ ]:
directed_df = generate_directed_graph(edges)
print(directed_df.head())

# Fill missing values with most frequent value

In [ ]:
print("missing values for columns:")
print(directed_df.isnull().sum())

print("filling missing values with most popular values per column!!!")
directed_df = directed_df.fillna({"disabled":False,"fee_base_msat":1000,"fee_rate_milli_msat":1,"min_htlc":1000})

for col in ["fee_base_msat","fee_rate_milli_msat","min_htlc"]:
    directed_df[col] = directed_df[col].astype("float64")
    
print(directed_df.dtypes)

# Aggregate weights for multiple channels between nodes

In [ ]:
grouped = directed_df.groupby(["src","trg","snapshot_id"])

In [ ]:
directed_aggr = grouped.agg({
    "channel_id":"nunique",
    "capacity":"sum",
    "fee_base_msat":"mean",
    "fee_rate_milli_msat":"mean"
})

In [ ]:
directed_aggr = directed_aggr.rename({"channel_id":"num_channels"}, axis=1)

### There are extreme cases of channel_id-s between nodes!!!

- are these attacks?

In [ ]:
directed_aggr.sort_values("num_channels",ascending=False).reset_index().head(20)

In [ ]:
directed_aggr = directed_aggr.reset_index()

In [ ]:
directed_aggr.head()

# Export

In [ ]:
output_dir = "/mnt/idms/fberes/data/bitcoin_ln_research/"

In [ ]:
directed_aggr.to_csv("%s/directed_temporal_edges.csv" % output_dir, index=False)